In [14]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from model.baseline_model import BaselineNonlinearModel
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [15]:
import matplotlib.pyplot as plt
from vecopsciml.operators.zero_order import Mx, My

In [16]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/sigmoid_nonlinear/sigmoid_nonlinear.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/sigmoid_nonlinear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/sigmoid_nonlinear/baseline')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/sigmoid_nonlinear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/sigmoid_nonlinear/baseline


In [17]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/sigmoid_nonlinear/sigmoid_nonlinear.pkl


In [18]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [19]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [20]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [21]:
n_modes = 10

# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, n_modes, 10, 20]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [22]:
# Load model and the optimizer
model = BaselineNonlinearModel(input_shape, predictive_layers, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 20000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 1.555e+09, Test loss: 1.781e+09, MSE(e): 1.111e+02, MSE(pi1): 4.380e+04, MSE(pi2): 4.659e+01, MSE(pi3): 4.977e+01
Epoch 100, Train loss: 9.858e+07, Test loss: 1.215e+08, MSE(e): 9.829e+00, MSE(pi1): 1.633e+01, MSE(pi2): 4.370e+00, MSE(pi3): 1.286e+00
Epoch 200, Train loss: 3.253e+07, Test loss: 3.797e+07, MSE(e): 3.208e+00, MSE(pi1): 3.235e+01, MSE(pi2): 1.660e+00, MSE(pi3): 1.275e+00
Epoch 300, Train loss: 1.832e+07, Test loss: 2.220e+07, MSE(e): 1.796e+00, MSE(pi1): 2.221e+01, MSE(pi2): 1.091e+00, MSE(pi3): 1.360e+00
Epoch 400, Train loss: 1.580e+07, Test loss: 1.945e+07, MSE(e): 1.546e+00, MSE(pi1): 1.913e+01, MSE(pi2): 9.948e-01, MSE(pi3): 1.417e+00
Epoch 500, Train loss: 1.491e+07, Test loss: 1.843e+07, MSE(e): 1.459e+00, MSE(pi1): 1.797e+01, MSE(pi2): 9.626e-01, MSE(pi3): 1.448e+00
Epoch 600, Train loss: 1.447e+07, Test loss: 1.784e+07, MSE(e): 1.414e+00, MSE(pi1): 1.747e+01, MSE(pi2): 9.467e-01, MSE(pi3): 1.462e+00
Epoch 700, 

In [25]:
# Parametros de entrenamiento
start_epoch = 18000
n_epochs = 100000

batch_size = 64
n_checkpoints = 5

second_lr = 3e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 18000.
Epoch 18000, Train loss: 1.879e+04, Test loss: 1.473e+05, MSE(e): 1.755e-03, MSE(pi1): 5.840e-02, MSE(pi2): 1.257e-03, MSE(pi3): 6.612e-03
Epoch 18100, Train loss: 3.605e+03, Test loss: 1.334e+05, MSE(e): 2.849e-04, MSE(pi1): 1.855e-02, MSE(pi2): 1.774e-04, MSE(pi3): 5.709e-03
Epoch 18200, Train loss: 3.589e+03, Test loss: 1.331e+05, MSE(e): 2.835e-04, MSE(pi1): 1.856e-02, MSE(pi2): 1.766e-04, MSE(pi3): 5.686e-03
Epoch 18300, Train loss: 3.579e+03, Test loss: 1.329e+05, MSE(e): 2.825e-04, MSE(pi1): 1.851e-02, MSE(pi2): 1.760e-04, MSE(pi3): 5.684e-03
Epoch 18400, Train loss: 3.570e+03, Test loss: 1.328e+05, MSE(e): 2.817e-04, MSE(pi1): 1.845e-02, MSE(pi2): 1.756e-04, MSE(pi3): 5.687e-03
Epoch 18500, Train loss: 3.562e+03, Test loss: 1.326e+05, MSE(e): 2.809e-04, MSE(pi1): 1.839e-02, MSE(pi2): 1.751e-04, MSE(pi3): 5.689e-03
Epoch 18600, Train loss: 3.555e+03, Test loss: 1.325e+05, MSE(e): 2.802e-04, MSE(pi1): 1.834e-02, MSE(pi2): 1.747e-0